In [70]:
from tensorflow.keras.datasets import imdb
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from tensorflow.keras.models import Model
import string
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/IMDB-Dataset.csv")
data['review'] = data['review'].str.lower()

In [4]:
data

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [6]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [7]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

In [8]:
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

In [9]:
data_without_stopwords.head()

,review,sentiment,review without stopwords,clean_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...


In [10]:
reviews = data_without_stopwords['clean_review']
reviews

0        one reviewers mentioned watching just 1 oz epi...
1        wonderful little production  the filming techn...
2        thought wonderful way spend time hot summer we...
3        basically family little boy  jake  thinks zomb...
4        petter mattei s  love time money  visually stu...
                               ...                        
49995    thought movie right good job  wasn t creative ...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    no one expects star trek movies high art  fans...
Name: clean_review, Length: 50000, dtype: object

In [11]:
reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])

In [12]:
sentiment = data_without_stopwords['sentiment']

In [17]:
data_without_stopwords['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [19]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))
y

array([1, 1, 1, ..., 0, 0, 0])

In [20]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)


In [21]:
len(X_train)

40000

In [22]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [23]:
words_to_index = tokenizer.word_index
len(words_to_index)

95419

In [24]:
words_to_index

{'movie': 1,
 'film': 2,
 's': 3,
 't': 4,
 'not': 5,
 'one': 6,
 'like': 7,
 'just': 8,
 'good': 9,
 'it': 10,
 'can': 11,
 'the': 12,
 'no': 13,
 'time': 14,
 'even': 15,
 'really': 16,
 'story': 17,
 'see': 18,
 'well': 19,
 'much': 20,
 'bad': 21,
 'get': 22,
 'great': 23,
 'people': 24,
 'will': 25,
 'also': 26,
 'first': 27,
 'don': 28,
 'made': 29,
 'make': 30,
 'way': 31,
 'i': 32,
 'movies': 33,
 'characters': 34,
 'think': 35,
 'character': 36,
 'watch': 37,
 'films': 38,
 'two': 39,
 'many': 40,
 'seen': 41,
 'love': 42,
 'plot': 43,
 'acting': 44,
 'never': 45,
 'life': 46,
 'best': 47,
 'show': 48,
 'know': 49,
 'this': 50,
 'little': 51,
 'ever': 52,
 'off': 53,
 'man': 54,
 'better': 55,
 'end': 56,
 'still': 57,
 'scene': 58,
 'say': 59,
 'and': 60,
 'scenes': 61,
 'something': 62,
 'go': 63,
 'back': 64,
 'real': 65,
 'now': 66,
 'thing': 67,
 'watching': 68,
 'actors': 69,
 'director': 70,
 'years': 71,
 'funny': 72,
 'didn': 73,
 'though': 74,
 'doesn': 75,
 'old': 7

In [30]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
  return word_to_vec_map

In [31]:
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/glove.6B.50d.txt')

In [32]:
maxLen = 150

In [33]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [34]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [36]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [37]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           4770950   
_________________________________________________________________
lstm (LSTM)                  (None, 150, 128)          91648     
_________________________________________________________________
dropout (Dropout)            (None, 150, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584

In [38]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           4770950   
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 512)          77312     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 47, 256)           393472    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 256)           1968

In [39]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [40]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [48]:
import tensorflow as tf 

In [49]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 42s 19ms/step - loss: 0.5595 - accuracy: 0.7075
Epoch 2/15
625/625 [==============================] - 12s 19ms/step - loss: 0.4637 - accuracy: 0.7826
Epoch 3/15
625/625 [==============================] - 12s 19ms/step - loss: 0.4137 - accuracy: 0.8105
Epoch 4/15
625/625 [==============================] - 12s 19ms/step - loss: 0.3846 - accuracy: 0.8270
Epoch 5/15
625/625 [==============================] - 12s 19ms/step - loss: 0.3501 - accuracy: 0.8465
Epoch 6/15
625/625 [==============================] - 12s 19ms/step - loss: 0.3325 - accuracy: 0.8565
Epoch 7/15
625/625 [==============================] - 12s 19ms/step - loss: 0.3077 - accuracy: 0.8669
Epoch 8/15
625/625 [==============================] - 12s 19ms/step - loss: 0.2826 - accuracy: 0.8823
Epoch 9/15
625/625 [==============================] - 12s 19ms/step - loss: 0.2547 - accuracy: 0.8943
Epoch 10/15
625/625 [==============================] - 12s 19ms/step - loss: 0.233

In [53]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 71s 105ms/step - loss: 0.5733 - accuracy: 0.6888
Epoch 2/15
625/625 [==============================] - 62s 100ms/step - loss: 0.4847 - accuracy: 0.7696
Epoch 3/15
625/625 [==============================] - 63s 100ms/step - loss: 0.4503 - accuracy: 0.7901
Epoch 4/15
625/625 [==============================] - 62s 100ms/step - loss: 0.4256 - accuracy: 0.8035
Epoch 5/15
625/625 [==============================] - 63s 100ms/step - loss: 0.4029 - accuracy: 0.8173
Epoch 6/15
625/625 [==============================] - 62s 99ms/step - loss: 0.3868 - accuracy: 0.8260
Epoch 7/15
625/625 [==============================] - 61s 98ms/step - loss: 0.3813 - accuracy: 0.8300
Epoch 8/15
625/625 [==============================] - 61s 98ms/step - loss: 0.3703 - accuracy: 0.8353
Epoch 9/15
625/625 [==============================] - 61s 97ms/step - loss: 0.3628 - accuracy: 0.8400
Epoch 10/15
625/625 [==============================] - 61s 98ms/step - loss: 

In [55]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [56]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 14s 42ms/step - loss: 0.3685 - accuracy: 0.8373


[0.3684707581996918, 0.8373000025749207]

In [57]:
model_1d.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.3805 - accuracy: 0.8298


[0.3805393874645233, 0.829800009727478]

In [58]:
preds = model_1d.predict(X_test_indices)

In [59]:
n = np.random.randint(0,9999)

X_test[n]

'early fulci work director shows mainstream side well talent compelling storytelling reasonable elucidation genre  personally think unfairly maligned throughout career aesthete gore  pretty capable procedural surprised subtextually rich narrative shows distrust small minded small town mentality inefficiency police  well twisted ideals catholic church  last seemed cut film knees first released possibly given director another direction early on  comparative lack gore film shows urgent psychological imperative film prevailing  mystery  gore puppetry utilise put good use notably scene falsely accused murderer senselessly lynched mob men graveyard left dead show vile antiquated vigilantism  expertly choreographed modern tunes off diegetic radio another example final scene  which indeed awesome  strikingly handsome priest falls cliff puppet used markedly showed medium closeup accentuated demonic appearance evident ugliness  curiously  considering later works  fulci also invokes idea modernit

In [60]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')

predicted sentiment : positive
correct sentiment : positive


In [61]:
preds[n]

array([0.9446634], dtype=float32)

In [62]:
Y_test[n]

1

In [63]:
model_1d.save_weights('/content/drive/My Drive/imdb_weights_con1vd.hdf5')


In [64]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)


In [65]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

In [66]:
data = add_score_predictions(data, reviews_list_idx)

In [67]:
data

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.657835,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.989694,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.988200,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.012905,negative
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.990067,positive
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.954781,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.002473,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.224869,negative
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.845549,positive
